<a href="https://colab.research.google.com/github/Bonapark101/Full-stack-data-analysis_Internship-Project/blob/main/Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Summary**

Cleaning and Standardizing data before analyzing historical sales performance of spot coffee lots alongside their attributes such as origin, varietal, process, SCA score, profile, and price.

***Objectives***
1. Identify which sales factors (timing, negotiated price, order pattern) and lot attributes (origin, varietal, process, crop year, SCA score, profile, bag size, initial price, etc.) are linked to good or bad performance
2. Learn from past spot coffee performance for successful selection of future specialty spot coffees


**Data Description**

This dataset combines coffee offerlists from Attesa Coffee. The final merged file contains information about coffee lots, origins, varietals, processes, sca score, and flavour profile.

**Column Overview**
lot_id = unique identifier for each coffee lot
origin = country of origin
Varietal = coffee variety
process = processing method
sca_score = quality score based on SCA cupping standard
flavour = flavour category from SCA standard

**Data Summary**
Time range: 2023 - 2025
Data source: PDF and Excel ZIP files provided by Attesa coffee


***NOTE**

I excluded the pricing data, because it is company's internal data.


### **Importing libraries**

In [ ]:
!pip install pdfplumber

import pdfplumber
import pandas as pd
from pathlib import Path
import zipfile


### **Importing PDF and Excel files**

**PDF and Excel files**


In [ ]:
# Convert PDF files into CSV
# Unzip files in a folder, convert each file

# Setting
zip_files = [Path("/content/22-23.zip"), Path("/content/pdf_dataset_25.zip")]
pdf_folder = Path("pdf_files")
csv_folder = Path("all_csvs")

pdf_folder.mkdir(exist_ok=True)
csv_folder.mkdir(exist_ok=True)

# Unzip PDFs
for zip_path_pdf in zip_files:
    with zipfile.ZipFile(zip_path_pdf, "r") as zip_ref:
        zip_ref.extractall(pdf_folder)

# Convert PDFs to CSVs
for pdf_path in pdf_folder.glob("*.pdf"):
    all_tables = []
    print(pdf_path.name)

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                df = pd.DataFrame(table)
                all_tables.append(df)

    if all_tables:
        pdf_df = pd.concat(all_tables, ignore_index=True)
        csv_name = pdf_path.stem + ".csv"
        pdf_df.to_csv(csv_folder / csv_name, index=False)
        print(csv_name)
    else:
        print(pdf_path.name)



In [ ]:

zip_path_excel = Path("/content/23-25.zip")
excel_folder = Path("excel_files")
csv_folder = Path("all_csvs")

excel_folder.mkdir(exist_ok=True)
csv_folder.mkdir(exist_ok=True)

# Unzip Excels
with zipfile.ZipFile(zip_path_excel, "r") as zip_ref:
    zip_ref.extractall(excel_folder)

# Convert Excel files to CSVs
for excel_path in excel_folder.glob("*.xls*"):
    print(excel_path.name)
    xls = pd.ExcelFile(excel_path)

    for sheet_name in xls.sheet_names:
        df = pd.read_excel(excel_path, sheet_name=sheet_name, dtype=str)
        csv_name = f"{excel_path.stem}_{sheet_name}.csv"
        df.to_csv(csv_folder / csv_name, index=False)
        print(csv_name)



### **Data cleaning**


Cleaning files

In [ ]:
#preview before cleaning
csv_folder = Path("/content/all_csvs")

for file in csv_folder.glob("*.csv"):
    print(f"\n📄 Previewing {file.name}")
    df = pd.read_csv(file, dtype=str)  # read everything as string because data in the files are inconsistent
    print(df.head())
    print(df.info())
    print(df.isna().sum())
    print(df.duplicated().sum())


In [ ]:
# Drop irrelevant columns and rows
csv_folder = Path("/content/all_csvs")
clean_folder = Path("/content/cleaned_csvs")
clean_folder.mkdir(exist_ok=True)

# Keywords to detect header row
keywords = ["Type", "Origin"]

# Columns you may want to drop
columns_to_drop = [
    'producer', 'status', 'crop_year', 'quantity', 'nan',
    'warehouse', 'crop\nyear', 'bags\navailable', 'bag_qty\n(kg)', 'availability'
]

skipped_files = []

# Loop through each CSV file
for file in csv_folder.glob("*.csv"):
    print(f"Processing {file.name}")

    #Detect header row
    df = pd.read_csv(file, dtype=str, header=None)
    header_row_idx = None
    for i, row in df.iterrows():
        if any(any(keyword.lower() in str(cell).lower() for keyword in keywords)
               for cell in row if pd.notna(cell)):
            header_row_idx = i
            break

    if header_row_idx is None:
        print(f"No header found in {file.name}, skipping")
        skipped_files.append(file.name)
        continue

    # Keep rows from header onward
    df = df.iloc[header_row_idx:].dropna(how='all').reset_index(drop=True)

    # Set the first row as header
    df.columns = df.iloc[0].astype(str).str.strip().str.lower().str.replace(' ', '_')
    df = df.iloc[1:].reset_index(drop=True)  # remove header row from data

    # Drop irrelevant columns
    df = df.drop(columns=columns_to_drop, errors='ignore')

    # Remove rows containing 'WAREHOUSE'
    df = df[~df.apply(lambda row: row.astype(str).str.contains('WAREHOUSE', case=False).any(), axis=1)]

    # Remove completely empty rows
    df = df.dropna(how='all').reset_index(drop=True)

    # Save cleaned CSV
    cleaned_path = clean_folder / file.name
    df.to_csv(cleaned_path, index=False)
    print(f"Saved cleaned CSV: {cleaned_path.name}")

    # Preview cleaned data
    print("First 5 rows:")
    print(df.head())
    print(df.info())
    print("Missing values per column:\n", df.isna().sum())
    print("Number of duplicate rows:", df.duplicated().sum())

    # List skipped files
if skipped_files:
    print("\nSkipped files (no header detected):")
    for f in skipped_files:
        print(f"- {f}")


In [ ]:
csv_folder = Path("/content/cleaned_csvs")

deleted_files = []

for csv_file in csv_folder.glob("*.csv"):
    if "2022" in csv_file.name:
        csv_file.unlink()  # delete the file
        deleted_files.append(csv_file.name)
        print(f"Deleted file: {csv_file.name}")

In [ ]:
# Inspect the processed files
csv_folder = Path("cleaned_csvs")

for csv_file in csv_folder.glob("*.csv"):
    print(f"\nPreview of {csv_file.name}")
    try:
        # Read the cleaned CSV
        df = pd.read_csv(csv_file, dtype=str)

        # Show first 5 rows
        print(df.head(5))

        # Column names and basic shape
        print(f"Columns: {list(df.columns)}")
        print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")

        # Check missing values and duplicates
        print("Missing values per column:\n", df.isna().sum())
        print("Number of duplicate rows:", df.duplicated().sum())

    except Exception as e:
        print(f"Could not read {csv_file.name}: {e}")


### **Standardization**

In [ ]:
# sca score and pricing-related columns have not properly transferred;
# so had to go through few attempts, but only went through for sca score, as pricing should not be publicly posted
csv_folder = Path("/content/cleaned_csvs")
standardized_folder = Path("/content/standardized_csvs")
standardized_folder.mkdir(exist_ok=True)

# Column mapping for other fields
column_mapping = {
    "Lot ID": "lot_id",
    "LotID": "lot_id",
    "Origin": "origin",
    "Country of Origin": "origin",
    "Type": "type",
    "Process": "process",
    "Varietal": "varietal",
    "Profile": "profile",
    "600+ kg": "over_600kg",
    "60+ kg": "over_60kg",
    "1.8+ tons": "over_1800kg",
    "3.6+ tons": "over_3600kg",
    "1+ bag": "over_60kg",
    "3+ bags": "over_180kg",
    "10+ bags": "over_600kg",
    "30+ bags": "over_1800kg",
    "30+ Bags": "over_1800kg",
    "60+ bags": "over_3600kg",
    "60+ Bags": "over_3600kg",
}

# Expected final schema
expected_columns = [
    "lot_id", "origin", "type", "process", "varietal",
    "sca_score", "profile", "over_60kg", "over_180kg",
    "over_600kg", "over_1800kg", "over_3600kg"
]

for csv_file in csv_folder.glob("*.csv"):
    df = pd.read_csv(csv_file, dtype=str)

    # Normalize column names: strip, lowercase, replace non-alphanumeric chars with underscore
    df.columns = [str(c).strip().lower().replace(' ', '_').replace('\\', '').replace('/', '') for c in df.columns]

    # Detect SCA score column automatically
    sca_col_candidates = [c for c in df.columns if 'sca' in c and 'score' in c]
    if sca_col_candidates:
        df = df.rename(columns={sca_col_candidates[0]: 'sca_score'})
    else:
        print(f"⚠️ No SCA score column found in {csv_file.name}")

    # Map other columns using column_mapping
    df = df.rename(columns=lambda x: column_mapping.get(x, x))

    # Add missing columns
    for col in expected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    # Reorder columns
    df = df[expected_columns]

    # Save standardized CSV
    standardized_path = standardized_folder / csv_file.name
    df.to_csv(standardized_path, index=False)
    print(f"Standardized {csv_file.name}")

print("✅ All files standardized and SCA scores preserved.")


In [ ]:
standardized_folder = Path("/content/standardized_csvs")

# Collect all standardized CSV files and merge them all
all_files = list(standardized_folder.glob("*.csv"))
combined = pd.concat(
    [pd.read_csv(f, dtype=str) for f in all_files],
    ignore_index=True
)

print(f"Merged {len(all_files)} standardized files — total rows: {len(combined)}")

# Optional: check structure
print("Columns:", list(combined.columns))
print("\nPreview of merged data:")
print(combined.head())


In [ ]:
# Save the merged file
output_path = Path("/content/final_merged_standardized.csv")
combined.to_csv(output_path, index=False)

print(f"\nFinal merged dataset saved successfully at: {output_path.resolve()}")


### **Data Standardization**

Standardizing_flavour profile

In [ ]:
# Standardize flavour profile according to SCA standard
import numpy as np
import re

df = pd.read_csv('/content/final_merged_standardized.csv')

# Define SCA inner-ring mapping
sca_inner_map = {
    'Fruity': ['blackberry', 'raspberry', 'blueberry', 'strawberry', 'raisin',
               'prune', 'coconut', 'cherry', 'pomegranate', 'pineapple',
               'grape', 'apple', 'peach', 'pear', 'grapefruit', 'orange', 'lemon', 'lime', 'mandarin'],
    'Sweet': ['brown sugar', 'vanilla', 'honey', 'caramelized', 'maple syrup', 'molasses', 'caramel'],
    'Nutty/Cocoa': ['peanut', 'almond', 'hazelnut', 'chocolate', 'dark chocolate', 'milk chocolate', 'cacao', 'cocoa'],
    'Spices': ['cinnamon', 'clove', 'anise', 'nutmeg', 'pepper', 'pungent'],
    'Roasted': ['cereal', 'burnt','tobacco', 'pipe tobacco', 'malt', 'grain', 'brown', 'roast', 'smoky', 'ashy', 'acrid'],
    'Floral': ['black tea', 'chamomile','jasmine', 'rose'],
    'Sour/Fermented': ['sour', 'alcohol', 'fermented', 'citric acid', 'malic acid', 'winey', 'whiskey', 'overripe'],
    'Green/Vegetative': ['olive oil', 'raw', 'green', 'vegetable', 'beany', 'under_ripe', 'peapod', 'fresh', 'herbal', 'hay', 'herb_like'],
    'Other': ['chemical', 'papery', 'musty', 'stale', 'cardboard', 'woody', 'moldy', 'earthy', 'animalic', 'meaty', 'brothy']
}

# Flatten valid terms
valid_terms = {term.lower() for group in sca_inner_map.values() for term in group}

# Clean and validate flavor profiles
def keep_valid_terms(profile):
    if pd.isna(profile):
        return np.nan
    # Split by commas, slashes, or semicolons
    words = re.split(r'[,/;]', profile.lower())
    words = [w.strip() for w in words if w.strip()]
    valid = [w for w in words if w in valid_terms]
    return ', '.join(sorted(set(valid))) if valid else np.nan

# Apply cleaning (keep all rows)
if 'profile' in df.columns:
    df['Profile_SCA_Valid'] = df['profile'].apply(keep_valid_terms)
else:
    raise KeyError("Column 'Profile' not found in the dataset. Please check your column names.")

# Preview result
print("Processed dataset — first 10 rows:")
print(df[['profile', 'Profile_SCA_Valid']].head(10))


In [ ]:
# Group the Profile_SCA_Valid values into SCA outer-ring profile
term_to_category = {term.lower(): category for category, terms in sca_inner_map.items() for term in terms}

# Function to map profile to standard_flavour
def map_to_standard_flavour(profile):
    if pd.isna(profile):
        return np.nan
    # Split by comma, slash, semicolon
    terms = [w.strip().lower() for w in re.split(r'[,/;]', profile)]
    # Map each term to its category
    categories = set(term_to_category.get(t) for t in terms if t in term_to_category)
    if categories:
        return ', '.join(sorted(categories))
    else:
        return np.nan

# Apply to the dataset
df['standard_flavour'] = df['Profile_SCA_Valid'].apply(map_to_standard_flavour)

# Inspect result
df[['Profile_SCA_Valid', 'standard_flavour']].head(20)


In [ ]:
# Remove duplicate rows based on 'lot_id', keep the first occurrence
df = df.drop_duplicates(subset=['lot_id'], keep='first').reset_index(drop=True)

# Preview result
print(f"After removing duplicates, total rows: {len(df)}")
df['lot_id'].head(10)


In [ ]:
#change data type_sca_score, over_60kg, over_180kg, over_600kg, over_600kg, over_1800kg, over_3600kg
# List of columns to convert
numeric_columns = [
    'sca_score',
    'over_60kg',
    'over_180kg',
    'over_600kg',
    'over_1800kg',
    'over_3600kg'
]

# Convert to float
for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Check the data types
print(df[numeric_columns].dtypes)


In [ ]:
# Keep only rows where lot_id is a string and starts with 'ATT'
mask = df['lot_id'].astype(str).str.strip().str.upper().str.startswith('ATT')
final_df = df[mask].reset_index(drop=True)

# Preview the first few rows to confirm
print(f"✅ Rows after filtering lot_id starting with 'ATT': {len(final_df)}")
print(final_df[['lot_id']].head())


In [ ]:
# Drop the 'type', 'over_60kg', 'over_180kg', 'over_600kg', 'over_1800kg', 'over_3600kg' column if it exists
columns_to_drop = ['type', 'over_60kg', 'over_180kg', 'over_600kg', 'over_1800kg', 'over_3600kg']

# Drop them if they exist
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    df = df.drop(columns=existing_columns_to_drop)
    print(f"Dropped columns: {existing_columns_to_drop}")
else:
    print("None of the specified columns were found.")



In [ ]:
# Save the file for analysis

# Define the path where you want to save
output_path = Path("/content/final_attesa.csv")

# Save to CSV
df.to_csv(output_path, index=False)

print(f"Final cleaned dataset saved to: {output_path}")
